In [ ]:
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
#print(transformers.__version__)
from datasets import load_dataset
import torch.nn as nn
from sentence_transformers import SentenceTransformer, losses, models,InputExample
from scipy.stats import pearsonr, spearmanr
from scipy.spatial import distance
from torch.utils.data import  DataLoader,random_split
from sklearn.model_selection import KFold
from tqdm.auto import tqdm 

In [ ]:
datasets_Train = load_dataset('glue', 'stsb', split = 'train')
datasets_Val = load_dataset('glue', 'stsb', split = 'validation')
datasets_Train = datasets_Train.remove_columns(['idx'])
datasets_Val = datasets_Val.remove_columns(['idx'])

datasets_Train = datasets_Train.add_column("label_div", [i/5 for i in datasets_Train['label']]) 
datasets_Val = datasets_Val.add_column("label_div", [i/5 for i in datasets_Val['label']]) 


In [ ]:
Val_data=[]
Val = pd.DataFrame()
Val_data.append(("sentence1",  [i for i in datasets_Val['sentence1']])) 
Val_data.append(("sentence2",  [i for i in datasets_Val['sentence2']]))
Val_data.append(("label_div", [i/5 for i in datasets_Val['label']]))
Val = pd.DataFrame.from_dict(dict(Val_data))


Train_data=[]
Train = pd.DataFrame()
Train_data.append(("sentence1",  [i for i in datasets_Train['sentence1']])) 
Train_data.append(("sentence2",  [i for i in datasets_Train['sentence2']]))
Train_data.append(("label_div", [i/5 for i in datasets_Train['label']]))
Train = pd.DataFrame.from_dict(dict(Train_data))
#Val['label_div'].values

In [ ]:
Train_to_loader= []
#Val = []

for row in tqdm(datasets_Train):
    Train_to_loader.append(InputExample(texts=[row['sentence1'],row['sentence2']],label=row['label_div']))
    

In [ ]:


def Conv_2_Dataloader(df, batch_size):
     return DataLoader(df, shuffle=True, batch_size=batch_size)

def predict_scores(Designer, User):
    return [1 - distance.cosine(Designer[i], User[i])
                  for i in range(User.shape[0])]

def evaluate(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted)**2))
    pearson = pearsonr(actual, predicted)[0]
    spearman = spearmanr(actual, predicted)[0]
    print("Pearson:", pearson)
    print("spearman:", spearman)
    print("RMSE:", rmse)
    return pearson,spearman, rmse

def model_evaluate(model,sent_1,sent_2, actual):
    s1 = model.encode(sent_1)
    s2 = model.encode(sent_2)
    scores = predict_scores(s1, s2)
    pearson,spearman, rmse = evaluate(actual, scores)
    return pearson,spearman, rmse




In [ ]:
x = 0
epoch = ['mean', 'cls','max','lasttoken','weightedmean']
model_ = ["sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1",
          #"princeton-nlp/sup-simcse-roberta-base",
          #"princeton-nlp/sup-simcse-bert-base-uncased",
          #"Contrastive-Tension/BERT-Base-CT-STSb",
          #"voidism/diffcse-roberta-base-sts",
          #"voidism/diffcse-bert-base-uncased-sts",
          #"kwang2049/TSDAE-askubuntu2nli_stsb",
          #"thenlper/gte-base"
          ]
for i in model_:
    for j in epoch:
        x +=1
        epochs = 5
        batch_size = 1
        results = []

        word_embedding_model = models.Transformer(i)
        pooling_model= models.Pooling(word_embedding_model.get_word_embedding_dimension(), j)
        model_new_ = SentenceTransformer(modules=[word_embedding_model, pooling_model])
        model = model_new_
        model.to('cuda')
        train_loss = losses.CosineSimilarityLoss(model=model)  #CosineSimilarityLoss  CoSENTLoss
        
        #train_df = data.iloc[train]
        #val_df = data.iloc[val]
        train_dataloader =  Conv_2_Dataloader(Train_to_loader, batch_size=batch_size)
        baseline_pearson,baseline_spearman, baseline_rmse = model_evaluate(model, Val['sentence1'].values, Val['sentence2'].values,
                                                    Val['label_div'].values
                                                    )
        model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=epochs, output_path=('/sbert_STS_b ' +j + ' '+ str(x)))

        train_pearson, train_spearman, train_rmse = model_evaluate(model, Train['sentence1'].values, Train['sentence2'].values,
                                                    Train['label_div'].values
                                                                  )
        
        val_pearson,val_spearman, val_rmse = model_evaluate(model, Val['sentence1'].values, Val['sentence2'].values,
                                                    Val['label_div'].values

                                                           )


        results.append({
            'train_pearson': train_pearson, 'train_spearman': train_spearman,'train_rmse': train_rmse,
            'val_pearson': val_pearson, 'val_spearman': val_spearman,'val_rmse': val_rmse,
            'baseline_pearson':baseline_pearson, 'baseline_spearman':baseline_spearman, 'baseline_rmse':baseline_rmse
        })